In [14]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer

train_data_path = 'train.tsv'
test_data_path = 'valid.tsv'
output_path = 'output.txt'
stop_path = 'stopwords.txt'

In [15]:
train_df= pd.read_csv(train_data_path,header=None,sep='\t',quoting=3)
train_x= train_df[2]
train_y= train_df[1]
stop= pd.read_csv(stop_path,header=None)[0]
stop=stop.astype(str).tolist()

In [16]:
stemmer = PorterStemmer()

def preprocess_text(text):
    words = text.lower().split()
    filtered_words = [stemmer.stem(word) for word in words if word not in stop]
    return filtered_words

train_x= train_x.apply(preprocess_text)

In [17]:
label_to_index = {'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}
index_to_label = {v: k for k, v in label_to_index.items()}


In [18]:
def create_feature_matrix(processed_texts, labels):
    vocabulary = set(word for text in processed_texts for word in text)
    vocab_size = len(vocabulary)
    
    word_to_index = {word: idx for idx, word in enumerate(vocabulary)}
    
    feature_matrix = np.zeros((len(processed_texts), vocab_size), dtype=int)
    class_count = np.zeros(len(set(labels)))

    for i, text in enumerate(processed_texts):
        for word in text:
            if word in word_to_index:
                feature_matrix[i, word_to_index[word]] += 1 
    
    encoded_labels = np.array([label_to_index[label] for label in labels])
    
    for label in labels:
        class_count[label_to_index[label]] += 1

    return feature_matrix, encoded_labels, class_count, word_to_index

X, Y, class_count, word_to_index= create_feature_matrix(train_x, train_y)

In [19]:
class MultinomialNaiveBayes:
    def __init__(self):
        self.class_priors = None
        self.feature_probs = None

    def fit(self, X, y):
        self.class_priors = np.log(np.bincount(y) / len(y))
        self.feature_probs = np.zeros((len(self.class_priors), X.shape[1]))
        
        for c in range(len(self.class_priors)):
            X_c = X[y == c]
            self.feature_probs[c] = (X_c.sum(axis=0) + 1) / (X_c.sum() + X.shape[1]) 

    def predict(self, X):
        eps = 1e-10 
        log_feature_probs = np.log(self.feature_probs.T.clip(eps, 1))
        
        log_probs = X @ log_feature_probs + self.class_priors
        print(log_probs)
        return np.argmax(log_probs, axis=1)

model = MultinomialNaiveBayes()
model.fit(X, Y)

predictions = model.predict(X)

accuracy = np.mean(predictions == Y)
print(f'Accuracy: {accuracy * 100:.2f}%')

[[ -79.20168302  -73.38481779  -76.42009859  -76.60680196  -79.23488158
   -78.66496927]
 [-130.76901125 -123.2872824  -122.85123868 -120.31100548 -124.14486163
  -125.09011402]
 [-113.60605559 -106.72131004 -105.32299042 -107.04734022 -104.89368197
  -107.74715963]
 ...
 [-141.04162105 -136.90589934 -135.44914128 -131.63397375 -137.79711004
  -136.6408239 ]
 [ -65.44113561  -62.09104149  -64.9220671   -65.77545005  -62.71106984
   -64.70611208]
 [-146.27093672 -150.76535457 -152.69511013 -152.93482643 -154.68076586
  -151.76385062]]
Accuracy: 73.92%


In [20]:
predicted_labels = [index_to_label[pred] for pred in predictions]
print(predicted_labels)
print(len(predicted_labels))
with open(output_path, 'w') as f:
    for label in predicted_labels:
        f.write(f"{label}\n")

['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'barely-true', 'half-true', 'true', 'barely-true', 'half-true', 'half-true', 'barely-true', 'mostly-true', 'mostly-true', 'false', 'barely-true', 'false', 'mostly-true', 'half-true', 'false', 'half-true', 'half-true', 'half-true', 'false', 'mostly-true', 'barely-true', 'true', 'true', 'true', 'pants-fire', 'half-true', 'false', 'half-true', 'false', 'pants-fire', 'false', 'half-true', 'pants-fire', 'false', 'pants-fire', 'false', 'barely-true', 'barely-true', 'false', 'half-true', 'barely-true', 'mostly-true', 'half-true', 'barely-true', 'false', 'mostly-true', 'half-true', 'mostly-true', 'true', 'mostly-true', 'false', 'pants-fire', 'true', 'false', 'barely-true', 'barely-true', 'true', 'half-true', 'mostly-true', 'half-true', 'mostly-true', 'true', 'false', 'barely-true', 'half-true', 'half-true', 'false', 

In [21]:
file= 'checker_files/multinomial_probas_train.npy'
given= np.load(file)
print(np.log(given))
argmax_indices = np.argmax(given, axis=1)
argmax_indices= [index_to_label[z] for z in argmax_indices]
print(argmax_indices)
print(len(argmax_indices))
print(argmax_indices==predicted_labels)

[[ -79.20168302  -73.38481779  -76.42009859  -76.60680196  -79.23488158
   -78.66496927]
 [-130.76901125 -123.2872824  -122.85123868 -120.31100548 -124.14486163
  -125.09011402]
 [-113.60605559 -106.72131004 -105.32299042 -107.04734022 -104.89368197
  -107.74715963]
 ...
 [-141.04162105 -136.90589934 -135.44914128 -131.63397375 -137.79711004
  -136.6408239 ]
 [ -65.44113561  -62.09104149  -64.9220671   -65.77545005  -62.71106984
   -64.70611208]
 [-146.27093672 -150.76535457 -152.69511013 -152.93482643 -154.68076586
  -151.76385062]]
['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'barely-true', 'half-true', 'true', 'barely-true', 'half-true', 'half-true', 'barely-true', 'mostly-true', 'mostly-true', 'false', 'barely-true', 'false', 'mostly-true', 'half-true', 'false', 'half-true', 'half-true', 'half-true', 'false', 'mostly-true', 'barely-true', 'true', 't

In [22]:
test_df= pd.read_csv(test_data_path,header=None,sep='\t',quoting=3)
test_x= train_df[2]
test_y= train_df[1]

In [23]:
def create_test_feature_matrix(processed_texts, word_to_index):
    vocab_size = len(word_to_index)
    
    feature_matrix = np.zeros((len(processed_texts), vocab_size), dtype=int)
    
    for i, text in enumerate(processed_texts):
        for word in text:
            if word in word_to_index:
                feature_matrix[i, word_to_index[word]] += 1  # Count occurrences of each word
    
    return feature_matrix

test_x= test_x.apply(preprocess_text)
X_test = create_test_feature_matrix(test_x, word_to_index)

test_predictions = model.predict(X_test)
test_predictions = [index_to_label[pred] for pred in test_predictions]
test_accuracy = np.mean(test_predictions == test_y)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


[[ -79.20168302  -73.38481779  -76.42009859  -76.60680196  -79.23488158
   -78.66496927]
 [-130.76901125 -123.2872824  -122.85123868 -120.31100548 -124.14486163
  -125.09011402]
 [-113.60605559 -106.72131004 -105.32299042 -107.04734022 -104.89368197
  -107.74715963]
 ...
 [-141.04162105 -136.90589934 -135.44914128 -131.63397375 -137.79711004
  -136.6408239 ]
 [ -65.44113561  -62.09104149  -64.9220671   -65.77545005  -62.71106984
   -64.70611208]
 [-146.27093672 -150.76535457 -152.69511013 -152.93482643 -154.68076586
  -151.76385062]]
Test Accuracy: 73.92%


In [24]:
print(test_predictions)

['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'barely-true', 'half-true', 'true', 'barely-true', 'half-true', 'half-true', 'barely-true', 'mostly-true', 'mostly-true', 'false', 'barely-true', 'false', 'mostly-true', 'half-true', 'false', 'half-true', 'half-true', 'half-true', 'false', 'mostly-true', 'barely-true', 'true', 'true', 'true', 'pants-fire', 'half-true', 'false', 'half-true', 'false', 'pants-fire', 'false', 'half-true', 'pants-fire', 'false', 'pants-fire', 'false', 'barely-true', 'barely-true', 'false', 'half-true', 'barely-true', 'mostly-true', 'half-true', 'barely-true', 'false', 'mostly-true', 'half-true', 'mostly-true', 'true', 'mostly-true', 'false', 'pants-fire', 'true', 'false', 'barely-true', 'barely-true', 'true', 'half-true', 'mostly-true', 'half-true', 'mostly-true', 'true', 'false', 'barely-true', 'half-true', 'half-true', 'false', 

In [25]:
print(test_y)

0              false
1          half-true
2        mostly-true
3              false
4          half-true
            ...     
10264    mostly-true
10265    mostly-true
10266      half-true
10267          false
10268     pants-fire
Name: 1, Length: 10269, dtype: object


In [26]:
valfile= 'checker_files/multinomial_probas_test.npy'
givenval= np.load(valfile)
print(np.log(givenval))
argmax_indices2 = np.argmax(given, axis=1)
argmax_indices2= [index_to_label[z] for z in argmax_indices2]
print(argmax_indices2)
print(len(argmax_indices2))
print(argmax_indices2==test_predictions)

[[ -24.9409246   -22.93496711  -22.61005719  -21.54883499  -21.38810705
   -22.64388665]
 [ -82.37630955  -80.82409254  -80.31749917  -80.13819655  -81.31237425
   -79.43371769]
 [-130.34163899 -123.64789376 -125.36535955 -124.90544465 -123.78190571
  -126.64799394]
 ...
 [ -67.10874758  -62.22740305  -62.70567967  -60.88907631  -62.45391265
   -61.77289397]
 [-187.09762558 -176.13803383 -177.94828144 -175.91503269 -179.23325708
  -177.60145899]
 [ -86.92133204  -78.86849058  -82.27759604  -80.65032896  -80.86006317
   -82.86644364]]
['false', 'half-true', 'mostly-true', 'false', 'half-true', 'true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'barely-true', 'half-true', 'true', 'barely-true', 'half-true', 'half-true', 'barely-true', 'mostly-true', 'mostly-true', 'false', 'barely-true', 'false', 'mostly-true', 'half-true', 'false', 'half-true', 'half-true', 'half-true', 'false', 'mostly-true', 'barely-true', 'true', 't